## Mesh Generation
### Strategy 1: Ball-Pivoting Algorithm

The idea behind the Ball-Pivoting Algorithm (BPA) is to simulate the use of a virtual ball to generate a mesh from a point cloud. We first assume that the given point cloud consists of points sampled from the surface of an object. Points must strictly represent a surface (noise-free), that the reconstructed mesh explicit.Using this assumption, imagine rolling a tiny ball across the point cloud “surface”. This tiny ball is dependent on the scale of the mesh, and should be slightly larger than the average space between points. When you drop a ball onto the surface of points, the ball will get caught and settle upon three points that will form the seed triangle. From that location, the ball rolls along the triangle edge formed from two points. The ball then settles in a new location: a new triangle is formed from two of the previous vertices and one new triangle is added to the mesh. As we continue rolling and pivoting the ball, new triangles are formed and added to the mesh. The ball continues rolling and rolling until the mesh is fully formed.

The idea behind the Ball-Pivoting Algorithm is simple, but of course, there are many caveats to the procedure as originally expressed here:
1. How is the ball radius chosen? The radius, is obtained empirically based on the size and scale of the input point cloud. In theory, the diameter of the ball should be slightly larger than the average distance between points.
2. What if the points are too far apart at some locations and the ball falls through? When the ball pivots along an edge, it may miss the appropriate point on the surface and instead hit another point on the object or even exactly its three old points. In this case, we check that the normal of the new triangle Facet is consistently oriented with the point's Vertex normals. If it is not, then we reject that triangle and create a hole.
3. What if the surface has a crease or valley, such that the distance between the surface and itself is less than the size of the ball? In this case, the ball would just roll over the crease and ignore the points within the crease. But, this is not ideal behavior as the reconstructed mesh is not accurate to the object.
4. What if the surface is spaced into regions of points such that the ball cannot successfully roll between the regions? The virtual ball is dropped onto the surface multiple times at varying locations. This ensures that the ball captures the entire mesh, even when the points are inconsistently spaced out.

1. First compute the necessary radius parameter based on the average distances computed from all the distances between points
2. Can then create a mesh and store it in the bpa_mesh variable
3. TODO: DETERMINE IF NEEDED: Before exporting the mesh, we can downsample the result to an acceptable number of triangles, for example, 100k triangles:



### Strategy 2: Poisson Reconstruction
The Poisson Reconstruction is a bit more technical/mathematical. Its approach is known as an implicit meshing method, which I would describe as trying to “envelop” the data in a smooth cloth. Without going into too many details, we try to fit a watertight surface from the original point set by creating an entirely new point set representing an isosurface linked to the normals. There are several parameters available that affect the result of the meshing:
1. Which depth? a tree-depth is used for the reconstruction. The higher the more detailed the mesh (Default: 8). With noisy data you keep vertices in the generated mesh that are outliers but the algorithm doesn’t detect them as such. So a low value (maybe between 5 and 7) provides a smoothing effect, but you will lose detail. The higher the depth-value the higher is the resulting amount of vertices of the generated mesh.
2. Which width? This specifies the target width of the finest level of the tree structure, which is called an octree 🤯. Don’t worry, I will cover this and best data structures for 3D in another article as it extends the scope of this one. Anyway, this parameter is ignored if the depth is specified.
3. Which scale? It describes the ratio between the diameter of the cube used for reconstruction and the diameter of the samples’ bounding cube. Very abstract, the default parameter usually works well.
4. Which fit? the linear_fit parameter if set to true, let the reconstructor use linear interpolation to estimate the positions of iso-vertices.

###### NOTES:
To get a clean result, it is often necessary to add a cropping step to clean unwanted artifacts. For this, we compute the initial bounding-box containing the raw point cloud, and we use it to filter all surfaces from the mesh outside the bounding-box:

The function output a list composed of an o3d.geometry object followed by a Numpy array. We want to select only the o3d.geometry justifying the [0] at the end.


### Export

Problems occurred with the format of the saved `off` files. It was noticed that they were saved as 

NOFF

|   a   |   b   |     c |d |e | f|g |
|-------|-------|-------|--|--|--|--|
| 8542  |  3283 |  0    |  |  |  |  |
|-65.053| 86.247|  83.61| 1| 0| 0| 1|
|-57.457| -5.34 | 83.611| 0| 0| 1| 1|  

This indicated that normals were being stored. This was easily turned off using the instructions from open3d. Moreover, `poission` additionally required not saving triangle colours.

# Mesh Generator

In [6]:
def normalise(pcd):
    """
    Computes the normals of a point cloud. Normals are oriented 
    with respect to the input point cloud if normals exist.
    Next, converts float64 numpy array of shape (n, 3) to Open3D format.
    Computed points are returned
    """
    radius = 0.1
    max_nn = 300

    try:
        pcd.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius,
                                                                                 max_nn),
                             fast_normal_computation=False)
        pcd.normals = o3d.utility.Vector3dVector(np.asarray(pcd.normals))
        return pcd
    except RuntimeError:
        print("Normals were not computed!")
        
        
def poission_mesh(pcd):
    """
    Computes and returns a triangle mesh from a oriented PointCloud pcd. 
    It implements the Screened Poisson Reconstruction proposed in 
    Kazhdan and Hoppe, "Screened Poisson Surface Reconstruction", 2013. 
    See https://github.com/mkazhdan/PoissonRecon
    Returns a cropped geometry based on an axis-aligned bounding box of 
    the geometry.
    """
    depth = 8
    width = 0
    scale = 1.1
    
    poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd,
                                                                             depth,
                                                                             width,
                                                                             scale,
                                                                             linear_fit=False)[0]
    bbox = pcd.get_axis_aligned_bounding_box()
    cropped_poission_mesh = poisson_mesh.crop(bbox)
    
    return cropped_poission_mesh


def save_obj(data_type, cropped_poisson_mesh, filename):
    """
    Save Meshes without respective normals in .obj format
    """
    o3d.io.write_triangle_mesh(output_path + 
                               data_type +
                               filename + "_mesh.off",
                               cropped_poisson_mesh)

    
def save_ply(data_type, cropped_poisson_mesh, filename):
    """
    Save Meshes without respective normals in .ply format
    """
    o3d.io.write_triangle_mesh(output_path + 
                               data_type +
                               filename + "_mesh.off",
                               cropped_poisson_mesh)

    
def save_off(choice, cropped_poisson_mesh, filename):
    """
    Save Meshes without respective normals in .off format
    """
    o3d.io.write_triangle_mesh(output_path + 
                               choice +
                               filename + "_mesh.off",
                               cropped_poisson_mesh,
                               write_vertex_normals=False,
                               write_vertex_colors=False)

    
def visualise(pcd):
    """
    Visualises pointcloud forms in a separate GUI:
        1. To visualise just pointclouds, pass `pcd`
        2. To visualise lod, choose between `poisson_lod_mix[100000000]` or
            `poisson_lod_mix[100000]`
    """
    print("Note: Visualisation opens in GUI")
    o3d.visualization.draw_geometries([pcd])
    
    
def lod_mesh_export(mesh, lods, extension, path):
    """
    Creates levels of detail and writes triangle meshes 
    to file
    """
    mesh_lods={}
    
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path  +"lod_" + str(i) + extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("Generation of " + str(i) + " LoD successful")
    
    return mesh_lods  


def noise_meshes_generator(choice):
    """
    Reads all noise xyz files and initialises point cloud objects.
    Calls helpers to generate, normalise and crop the pointcloud meshes
    Saves the meshes in desired format
    
    ***Available Formats:**
    .off : As required for PointNet
    .obj
    .ply
    """
    print("Starting mesh generation for noise xyz files...")
    
    for file in glob.glob(input_path + choice + "*.xyz"):
        point_cloud = np.loadtxt(file)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud) 
        
        normalised_pcd = normalise(pcd)
        cropped_poisson_mesh = poission_mesh(normalised_pcd)
        
        filename = os.path.splitext(os.path.basename(file))[0]
    
        {'obj': save_obj, 
         'ply': save_ply, 
         'off': save_off}['off'](choice,
                                 cropped_poisson_mesh,
                                 filename)
          
    print("Generated and saved all meshes in {0}".format(output_path))
    
    
def mixed_meshes_generator(choice):
    """
    Reads all mixed xyz files and initialises point cloud objects.
    Calls helpers to generate, normalise and crop the pointcloud meshes
    Saves the meshes in desired format
    
    ***Available Formats:**
    .off : As required for PointNet
    .obj
    .ply
    """
    print("Starting mesh generation for noise and hits xyz files...")

    for file in glob.glob(input_path + choice + "*.xyz"):
        point_cloud = np.loadtxt(file)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(point_cloud)
        
        normalised_pcd = normalise(pcd)
        cropped_poisson_mesh = poission_mesh(normalised_pcd)
        
        filename = os.path.splitext(os.path.basename(file))[0]

        {'obj': save_obj, 
         'ply': save_ply, 
         'off': save_off}['off'](choice,
                                 cropped_poisson_mesh,
                                 filename)
    print("Generated and saved all meshes in {0}".format(output_path))

In [7]:
import numpy as np
import pandas as pd
import open3d as o3d
import os
import glob

input_path = "../../../data/unsampled/xzt/processed_points/"
output_path = "../../../data/unsampled/xzt/meshes/"
data_choices = ["mixed/"]
data_types = ["mixed" ]


for choice, types in zip(data_choices, data_types):
    print(choice, types)
    {'noise': noise_meshes_generator,
     'mixed': mixed_meshes_generator}[types](choice)

mixed/ mixed
Starting mesh generation for noise and hits xyz files...
Generated and saved all meshes in ../../../data/unsampled/xzt/meshes/


In [3]:
pre_files = []
post_files = []
for file in glob.glob(input_path + 'mixed/' + "*.xyz"):
    pre_files.append(os.path.splitext(os.path.basename(file))[0])

    
for file in glob.glob(output_path + 'mixed/' + "*.off"):
    base = os.path.splitext(os.path.basename(file))[0].split('_',2)[0]
    group = os.path.splitext(os.path.basename(file))[0].split('_',2)[1]
    post_files.append(base+'_'+group)

In [4]:
# np.setdiff1d(post_files,pre_files)
remaining_files = np.setdiff1d(pre_files, post_files)

In [5]:
choice = 'mixed/'
for nfile in remaining_files:
    file = input_path + choice + nfile + ".xyz"
    print(input_path + choice + nfile + ".xyz")
    point_cloud = np.loadtxt(file)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
        
    normalised_pcd = normalise(pcd)
    cropped_poisson_mesh = poission_mesh(normalised_pcd)
        
    filename = os.path.splitext(os.path.basename(file))[0]

    {'obj': save_obj,
     'ply': save_ply,
     'off': save_off}['off'](choice,
                             cropped_poisson_mesh,
                             filename)
    print("Generated and saved all meshes in {0}".format(output_path))

../../../data/unsampled/xzt/processed_points/noise/group_3352.xyz
Generated and saved all meshes in ../../../data/unsampled/xzt/meshes/
../../../data/unsampled/xzt/processed_points/noise/group_3355.xyz
Generated and saved all meshes in ../../../data/unsampled/xzt/meshes/
../../../data/unsampled/xzt/processed_points/noise/group_3440.xyz
Generated and saved all meshes in ../../../data/unsampled/xzt/meshes/
../../../data/unsampled/xzt/processed_points/noise/group_347.xyz
Generated and saved all meshes in ../../../data/unsampled/xzt/meshes/
../../../data/unsampled/xzt/processed_points/noise/group_3487.xyz
Generated and saved all meshes in ../../../data/unsampled/xzt/meshes/
../../../data/unsampled/xzt/processed_points/noise/group_3492.xyz
Generated and saved all meshes in ../../../data/unsampled/xzt/meshes/
../../../data/unsampled/xzt/processed_points/noise/group_3569.xyz
Generated and saved all meshes in ../../../data/unsampled/xzt/meshes/
../../../data/unsampled/xzt/processed_points/nois